<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#План-работы" data-toc-modified-id="План-работы-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>План работы</a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-данных-и-подключение-библиотек" data-toc-modified-id="Загрузка-данных-и-подключение-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка данных и подключение библиотек</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Машинное-обучение-без-нейросети.-Логистическая-регрессия" data-toc-modified-id="Машинное-обучение-без-нейросети.-Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Машинное обучение без нейросети. Логистическая регрессия</a></span><ul class="toc-item"><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Предобработка данных</a></span></li><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#XGBClassifier" data-toc-modified-id="XGBClassifier-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>XGBClassifier</a></span></li><li><span><a href="#SGDClassifier" data-toc-modified-id="SGDClassifier-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>SGDClassifier</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества `F1` не меньше `0.75`. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять `BERT` необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`.  

Столбец `text`  — в нём содержит текст комментария, а `toxic` — целевой признак.


### План работы

1. Проведу предварительную обработку данных
2. Обучу несколько моделей и протестирую их. Сначала проведу обучению без нейросетки, а затем применю BERT.
3. Сравню метрику качества `f1` (должен быть не меньше `0.75`).
4. Сделаю выводы. 

## Подготовка

### Загрузка данных и подключение библиотек

In [1]:
#%pip install torch
#%pip install tensorflow # установилась с 1 ошибкой
#%pip install transformers
#%pip install pymystem3
#%pip install textblob
#%pip install xgboost
#%pip install bert-for-tf2
#%pip install sentencepiece
#%pip install tensorflow
#%pip install tensorflow_hub

#%matplotlib inline
#%pip install scikit-learn --user --upgrade pip

In [2]:
#update sklearn
#%pip install --upgrade sklearn

In [3]:
import numpy as np
import pandas as pd
import time

from torch import cuda
from torch import Tensor

import nltk # Natural Language Toolkit, «инструментарий естественного языка»
from nltk.corpus import stopwords as nltk_stopwords # корпус топ-слов
from sklearn.feature_extraction.text import CountVectorizer # счётчик слов для создания векторов
from sklearn.feature_extraction.text import TfidfVectorizer # счётчик величин TF-IDF 
#from pymystem3 import Mystem # для лемматизации русских слов
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from textblob import TextBlob, Word
from sklearn.pipeline import Pipeline

# для BERT
import torch
import transformers
from tqdm import notebook
import re
from tqdm import tqdm

# импорт метрики качества 
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, RandomizedSearchCV

# импорт моделей 
from sklearn.linear_model import LogisticRegression
#from catboost import CatBoostClassifier
#from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
#from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import SGDClassifier

import warnings
warnings.filterwarnings('ignore')

In [4]:
#КОД РЕВЬЮЕРА
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.model_selection import HalvingRandomSearchCV

In [5]:
# скачаем стоп-слова
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\eabdurakhmanova\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eabdurakhmanova\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eabdurakhmanova\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\eabdurakhmanova\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [6]:
# чтение данных
try:
    data = pd.read_csv('/datasets/toxic_comments.csv')
except:
    data = pd.read_csv('toxic_comments.csv')

In [7]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


**Краткий вывод** 

В данных два столбца: 
- `text` с комментариями (на английском языке)
- `toxic` с метками является ли комментарий позитивным (1) или нет (0). 

Данные полные без пропусков. 

## Обучение

### Машинное обучение без нейросети. Логистическая регрессия 

Обучу логистическую регрессию определять тональность текста. 

Разделю выборку на тренировочную и тестовую. 

Сначала лемматизирую тексты. Затем подсчитаю величину `TF-IDF` (оценим важность слов) для текстов. 

Обучу логистическую регрессию определять результаты предсказания для тестовой выборки комментариев.

Найду значение `f1-score`, которое должно быть не меньше `0.75`. 

#### Предобработка данных 

In [9]:
# очистка текста от лишних символов
# код взят отсюда https://stackabuse.com/text-classification-with-bert-tokenizer-and-tf-2-0-in-python/

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)
    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    return sentence

In [10]:
#data['text'] = data['text'].apply(lambda x: preprocess_text(x))

In [11]:
from multiprocessing import Pool

In [12]:
tqdm.pandas()

if __name__ == '__main__':
    with Pool(4) as p:
        data['text'] = data['text'].progress_apply(preprocess_text) 
        

100%|███████████████████████████████████████████████████████████████████████| 159571/159571 [00:13<00:00, 12052.22it/s]


In [13]:
# разделение выборки на обучающую и тестовую
state = np.random.RandomState(12345)

features = data['text']
target = data['toxic']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.3, random_state=state)

Код ниже я нашла по ссылке https://webdevblog.ru/podhody-lemmatizacii-s-primerami-v-python/ 

Функция ниже нужна для того, чтобы лемматизация проходила корректно. 

In [14]:
def lemmatize_with_postag(sentence):
    sent = TextBlob(sentence)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [15]:
%%time
# Lemmatize
#lemmatized_output = X_train.apply(lambda x: lemmatize_with_postag(x))
tqdm.pandas()


if __name__ == '__main__':
    with Pool(32) as p:
        lemmatized_output = X_train.progress_apply(lemmatize_with_postag) 

100%|█████████████████████████████████████████████████████████████████████████| 111699/111699 [10:26<00:00, 178.36it/s]


Wall time: 10min 29s


In [16]:
lemmatized_output.head()

5982      think Access Denied be perfectly valid usernam...
155963    Please stop abuse the word hopefully Ignorant ...
57212     Progressive Dance music see you don surrender ...
125660    Japanese character They annoy me this be an en...
105510    Yeah it do seem kind of over and just to let y...
Name: text, dtype: object

In [17]:
%%time
# создадим корпус лемм для тестовой выборки
#lemmatized_output_test = X_test.apply(lambda x: lemmatize_with_postag(x))

tqdm.pandas()


if __name__ == '__main__':
    with Pool(32) as p:
        lemmatized_output_test = X_test.progress_apply(lemmatize_with_postag) 


100%|███████████████████████████████████████████████████████████████████████████| 47872/47872 [04:32<00:00, 175.60it/s]


Wall time: 4min 34s


In [18]:
# создам матрицу cо значениями TF-IDF по корпусу твитов
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 
tf_idf_fit = count_tf_idf.fit(lemmatized_output) # матрица
tf_idf = tf_idf_fit.transform(lemmatized_output)
tf_idf_test = tf_idf_fit.transform(lemmatized_output_test) # берем count_tf_idf с обучающей выборки

# выводим размеры матрицы
print("Размер матрицы:", tf_idf.shape)
print("Размер матрицы:", tf_idf_test.shape)

Размер матрицы: (111699, 130550)
Размер матрицы: (47872, 130550)


In [19]:
from scipy.sparse import csr_matrix
tf_idf[:10].todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

Ну что ж... ))) Данные для анализа готовы. Перейдем к обучению моделей.

Здесь я обучу 3 модели:
- логистическую регрессию
- XGBClassifier
- SGDClassifier

Я также рассматривала модели GradientBoostingClassifier, CatBoostClassifier и LGBMClassifier, но они очень долго считались. Не стала использовать их в проекте 

Сравню время обучения, предсказания и качество метрики `f1-score`. 


#### Логистическая регрессия

In [20]:
start_time = time.time()

logistic_model = LogisticRegression(random_state=state, 
                        class_weight="balanced")

logistic_model.fit(tf_idf, y_train)
target_predicted = logistic_model.predict(tf_idf)

print("F1-score для тренировочной выборки равен", f1_score(y_train, target_predicted))

logistic_model_fit_time = time.time() - start_time

F1-score для тренировочной выборки равен 0.8300341296928329


In [21]:
%%time

# Проверим качество модели на кросс-валидации
cv_f1_logistic_model = (cross_val_score(logistic_model, 
                             tf_idf, 
                             y_train, 
                             cv=5, 
                             n_jobs=4,
                             scoring='f1').mean())
print('F1-score на кросс-валидации равен', cv_f1_logistic_model)

F1-score на кросс-валидации равен 0.7435195893621628
Wall time: 22.1 s


In [22]:
# время предсказания
start_time = time.time()

target_test_predicted = logistic_model.predict(tf_idf_test)
logistic_model_test_f1 = f1_score(y_test, target_test_predicted)

print("F1-score для тестовой выборки равен", logistic_model_test_f1)

logistic_model_predicted_time = time.time() - start_time

F1-score для тестовой выборки равен 0.7451858486341244


**Краткий вывод** 

Наша модель переобучилась на тренировочных данных. Но кросс-валидация спасает ситуацию.`F1-score` на кросс-валидации и на тестовой выборке примерно одинаковые. И мы практически достигли цели, но `f1-score` немного меньше `0.75`. Попробуем подобрать гиперпараметры для логистической регресси и проверим измениться ли качество метрики.

In [23]:
%%time
parameters = parameters = {'C': [3, 5, 10],
                           'class_weight':['balanced', None]}

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=state)

# заменила GridSearchCV 
grid_search = RandomizedSearchCV(logistic_model, 
                           parameters, 
                           n_jobs=4, 
                           scoring="f1", 
                           cv=skf) # cv=skf
grid_search = grid_search.fit(tf_idf, y_train)
print('Лучшие гиперпараметры для логистической регресси', grid_search.best_estimator_)
print('Лучший показатель f1-score', grid_search.best_score_)

f1_grid_search = grid_search.best_score_

Лучшие гиперпараметры для логистической регресси LogisticRegression(C=10, class_weight='balanced',
                   random_state=RandomState(MT19937) at 0x2407290FE40)
Лучший показатель f1-score 0.7612000441938818
Wall time: 1min 55s


In [24]:
# время предсказания
start_time = time.time()

target_test_predicted = grid_search.predict(tf_idf_test)
logistic_model_test_f1 = f1_score(y_test, target_test_predicted)

print("F1-score для тестовой выборки равен", logistic_model_test_f1)

logistic_model_predicted_time = time.time() - start_time

F1-score для тестовой выборки равен 0.7577992058990358


**Краткий вывод**

Круто! При подборе гиперпараметра `C` удалось повысить качество модели. `F1-score` не меньше `0.75`. 

Обучим некоторые более сложные модели и оценим метрику для них.

In [27]:
%%time

# время обучения
start_time = time.time()

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=stopwords)),
    ('logreg', LogisticRegression(random_state=state)),
])
parameters = {
    'tfidf__max_df': (0.5, 0.75), # удаляем очень частые слова
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)], # для N-грамм
    'logreg__C': (0.0001, 0.001, 0.01, 0.1, 1, 10),
    'logreg__class_weight':['balanced', None]
}

# Изменения 
grid_search_tune = RandomizedSearchCV(pipeline, 
                                      parameters, 
                                      cv=skf, 
                                      n_jobs=4, 
                                      scoring='f1', 
                                      verbose=3)

grid_search_tune.fit(lemmatized_output, y_train)

grid_search_tune_fit_time = time.time() - start_time

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Wall time: 1h 1min 39s


In [29]:
grid_search_tune.best_params_

{'tfidf__ngram_range': (1, 3),
 'tfidf__max_df': 0.75,
 'logreg__class_weight': None,
 'logreg__C': 10}

In [30]:
cv_f1_logistic_model = grid_search_tune.best_score_
print(cv_f1_logistic_model)

0.76312075781823


In [31]:
%%time

# время предсказания
start_time = time.time()
grid_search_tune_test_pred = grid_search_tune.predict(lemmatized_output_test)

grid_search_tune_test_f1 = f1_score(y_test, grid_search_tune_test_pred)

print("F1-score для тестовой выборки равен", grid_search_tune_test_f1)

grid_search_tune_predicted_time = time.time() - start_time

F1-score для тестовой выборки равен 0.7789379140002246
Wall time: 17.6 s
Parser   : 149 ms


**Краткий вывод** 

Обучение проходит сравнительно недолго. 
Качество модели улучшилось, на тестовой выборке `f1-score` = `0.77`. И оно сопоставимо с результатом, где логистическая модель с гиперпараметрами обучалась без pipeline.


#### XGBClassifier



In [32]:
%%time

start_time = time.time()
xgb_cl = XGBClassifier()

# подберем гиперпараметры 
params_xgb = {
    'max_depth': range (2, 10, 2),
    'n_estimators': range(50, 100, 10),
    #'learning_rate': [0.1, 0.01, 0.05] # уменьшила число подбираемых гиперпараметров, потому что долго считалось
}

# заменила GridSearchCV 
xgb_cl_gsc = RandomizedSearchCV(xgb_cl, 
                            params_xgb, 
                           n_jobs=4, 
                           scoring="f1", 
                           cv=skf)

xgb_cl_gsc = xgb_cl_gsc.fit(tf_idf, y_train)

cv_f1_xgb_model = xgb_cl_gsc.best_score_

print("F1-score для тренировочной выборки равен", cv_f1_xgb_model)

xgb_cl_fit_time = time.time() - start_time

F1-score для тренировочной выборки равен 0.71847553388331
Wall time: 31min 19s
Parser   : 576 ms


In [33]:
# время предсказания 
start_time = time.time()

target_xgb_cl_pred = xgb_cl_gsc.predict(tf_idf_test)
target_xgb_cl_test_f1 = f1_score(y_test, target_xgb_cl_pred)

print("F1-score для тестовой выборки равен", target_xgb_cl_test_f1)

xgb_cl_predicted_time = time.time() - start_time

F1-score для тестовой выборки равен 0.7304432065875515


**Краткий вывод** 

Без кросс-валидации модель переобучилась, `f1-score` сильно выше, чем на тестовой выборке. Результаты кросс-валидации и на тестовой выборке примерно одинаковые. Но `f1-score` не достигает нужного значения и ниже, чем результат логистической регрессии. Перейдем к следующей модели.


#### SGDClassifier

In [34]:
%%time

start_time = time.time()

sgd_pipeline = Pipeline([
               ('tfidf', TfidfVectorizer(stop_words=stopwords)),
               ('clf', SGDClassifier(random_state=12345))
])

parameters = {
    'tfidf__max_df': (0.5, 0.75), # удаляем очень частые слова
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)], # для N-грамм
    'clf__penalty': ['l2', 'l1', 'elasticnet'],
    'clf__class_weight': ['balanced', None],
    'clf__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10]
}


gs_clf = RandomizedSearchCV(sgd_pipeline, 
                            parameters,
                            cv=skf, 
                            n_jobs=4, 
                            scoring='f1',
                            verbose=3,
                            random_state = 12345)

gs_clf = gs_clf.fit(lemmatized_output, y_train)

sgd_cl_fit_time = time.time() - start_time

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Wall time: 12min 28s


In [35]:
cv_f1_sgd_cl_model = gs_clf.best_score_
print(cv_f1_sgd_cl_model)

0.6106084872361416


In [36]:
# сохраним модель с лучшими параметрами и обучим её
start_time = time.time()

# проверим сколько времени требуется для предсказания и проверим качество модели на тестовой выборке
target_sgd_pred = gs_clf.predict(lemmatized_output_test)

sgd_cl_test_f1 = f1_score(y_test, target_sgd_pred)

print("F1-score для тестовой выборки равен", sgd_cl_test_f1)

sgd_cl_model_predicted_time = time.time() - start_time

F1-score для тестовой выборки равен 0.6181573638197713


**Краткий вывод**

Как-то результаты ухудшились.


#### Общий вывод



In [37]:
pd.options.display.float_format = '{:,.2f}'.format

index = ['LogisticRegression (pipeline)',
         'XGBClassifier',
         'SGDClassifier']
data_score = {'F1 на RandomizedSearchCV':[cv_f1_logistic_model,
                                 cv_f1_xgb_model,
                                 cv_f1_sgd_cl_model],
        'F1 модели на тестовой выборке':[grid_search_tune_test_f1,
                                         target_xgb_cl_test_f1,
                                         sgd_cl_test_f1],
        'Время обучения модели, сек':[grid_search_tune_fit_time,
                                      xgb_cl_fit_time, 
                                      sgd_cl_fit_time],
        'Время предсказания модели, сек':[grid_search_tune_predicted_time,
                                          xgb_cl_predicted_time,
                                          sgd_cl_model_predicted_time]}

scores_data = pd.DataFrame(data=data_score, index=index)

display(scores_data)

,F1 на RandomizedSearchCV,F1 модели на тестовой выборке,"Время обучения модели, сек","Время предсказания модели, сек"
LogisticRegression (pipeline),0.76,0.78,"3,699.52",17.61
XGBClassifier,0.72,0.73,"1,879.47",0.67
SGDClassifier,0.61,0.62,748.38,14.93


Итак, логистическая регрессия показывает лучший результат. 

## Выводы

В рамках этого проекта была проведена классификация комментарии на позитивные и негативные. 

Для этого исходные тексты были очищены от несимвольных знаков, стоп-слов, проведена лемматизация и обучены три модели. Лучший показатель `f1-score` (`0.76`) был достигнут при помощи логистической регрессии.
